In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jul 19 19:29:30 2018

@author: karthikranjan
"""
import numpy as np
import re
import pickle 
import nltk
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
import time
import heapq

In [2]:
# Unpickling  pre-processed dataset

X_in = open('PreProcessedData.pickle','rb')
y_in = open('y.pickle','rb')
z_in = open('z.pickle','rb')
X = pickle.load(X_in)
y = pickle.load(y_in)
z = pickle.load(z_in)

In [3]:
import numpy
# fix random seed for reproducibility
numpy.random.seed(777)

In [4]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
text_train, text_test, sent_train, sent_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [5]:
# Creating the Tf-Idf Model --Filter Model approach

transformer = TfidfTransformer(sublinear_tf=True)
X = transformer.fit_transform(X).toarray()
text_train = transformer.fit_transform(text_train).toarray()
text_test = transformer.transform(text_test).toarray()

In [6]:
# Shape of train and test documents

print("No. of News Documents: ", X.shape[0])
print("No. of train Documents: ", text_train.shape[0])
print("No. of test Documents: ", text_test.shape[0])
print("No. of Input features: ", text_train.shape[1])

No. of News Documents:  4489
No. of train Documents:  3591
No. of test Documents:  898
No. of Input features:  11810


In [9]:
# importing the keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

from sklearn.metrics import accuracy_score

C:\Users\karthikranjan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [56]:
#### K- fold -validation for filter model ##########

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(Dense(units=1024, activation='relu', input_shape=(11810,)))

    # Add fully connected layer with a ReLU activation function
    network.add(Dense(units=512, activation='relu'))

    # Add fully connected layer with a sigmoid activation function
    network.add(Dense(units=5, activation='sigmoid'))

    # Compile neural network
    network.compile(loss='sparse_categorical_crossentropy', # Cross-entropy
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['accuracy']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [57]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
# Wrap Keras model so it can be used by scikit-learn
neural_network = KerasClassifier(build_fn=create_network, 
                                 epochs=5, 
                                 batch_size=100, 
                                 verbose=1)

In [58]:
# Evaluate neural network using three-fold cross-validation
val_score_filter  = cross_val_score(neural_network, X, y, cv=5)

Epoch 1/5
3591/3591 [==============================] - 15s 4ms/step - loss: 1.0862 - acc: 0.6734
Epoch 2/5
3591/3591 [==============================] - 13s 4ms/step - loss: 0.1801 - acc: 0.9373
Epoch 3/5
3591/3591 [==============================] - 13s 4ms/step - loss: 0.0682 - acc: 0.9724
Epoch 4/5
3591/3591 [==============================] - 13s 4ms/step - loss: 0.0607 - acc: 0.9727
Epoch 5/5
898/898 [==============================] - 1s 2ms/step
Epoch 1/5
3591/3591 [==============================] - 15s 4ms/step - loss: 1.0396 - acc: 0.6970
Epoch 2/5
3591/3591 [==============================] - 13s 4ms/step - loss: 0.1689 - acc: 0.9357
Epoch 3/5
3591/3591 [==============================] - 13s 4ms/step - loss: 0.0728 - acc: 0.9708
Epoch 4/5
3591/3591 [==============================] - 13s 4ms/step - loss: 0.0613 - acc: 0.9749
Epoch 5/5
898/898 [==============================] - 1s 1ms/step
Epoch 1/5
3591/3591 [==============================] - 15s 4ms/step - loss: 1.0574 - acc: 0.70

In [61]:
print("5-fold cross validation of an Filter Model scores:", val_score_filter)

print("5-fold cross validation of an Filter Model Average scores:", np.average(val_score_filter))

5-fold cross validation of an Filter Model scores: [0.88530067 0.86414253 0.87416481 0.8674833  0.86176143]
5-fold cross validation of an Filter Model Average scores: 0.8705705473260584


In [7]:
# Import PySwarms
import pyswarms as ps

# importing the keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

from sklearn.metrics import accuracy_score

C:\Users\karthikranjan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
from sklearn.neural_network import MLPClassifier

In [9]:
# Define objective function
def f_per_particle(m, alpha, count):
    
    total_features = 11810
    
    print("Count of particles", count)
    count = count+1
    
    # Get the subset of the features from the binary mask
    if np.count_nonzero(m) == 0:
        X_subset = text_train
        #print("%f In if statement" , X_subset)
    else:
        #X_subset = X[:,(np.logical_or(m > 0 , m == 1))]
        X_subset = text_train[:, m > 0]
        #print("%f In else statement" , X_subset)
       
        
        
    #units = int((5 + X_subset.shape[1]) / 2)
    clf = MLPClassifier(hidden_layer_sizes=(1024,512), max_iter=5, alpha=0.0001, solver='adam', verbose=2,  random_state=21,tol=0.000000001)


    # Fit the model
    clf.fit(X_subset, sent_train)
    
    ypred = clf.predict(X_subset)
    
    P = accuracy_score(sent_train, ypred)
    #print("P value", P)
    
    # Compute for the objective function
    j = (alpha * (1.0 - P)
        + (1.0 - alpha) * (1 - (X_subset.shape[1] / total_features)))

    return j

def f(x, alpha=0.88):
   
    n_particles = x.shape[0]
    count = 0
    j = [f_per_particle(x[i], alpha, count+i) for i in range(n_particles)]
    
    #for i in range(n_particles):
        #print("position of i", i)
        #j = [f_per_particle(x[i], alpha)]
        
    return np.array(j)

In [10]:
# Initialize swarm, arbitrary
options = {'c1': 2, 'c2': 2, 'w':0.9, 'k': 30, 'p':2}

# Call instance of PSO
dimensions = 11810 # dimensions should be the number of features

optimizer = ps.discrete.BinaryPSO(n_particles=35, dimensions=dimensions, options=options)

# start time
start_time = time.clock()
#f = f(X, 0.88)
# Perform optimization
cost, pos = optimizer.optimize(f, print_step=5, iters=10, verbose=2)

# end time
end_time = time.clock()

total_time_BPSO = (end_time - start_time)

with open('pos_ann_updated.pickle','wb') as f:
    pickle.dump(pos,f)

Count of particles 0
Iteration 1, loss = 1.47925413
Iteration 2, loss = 0.74458040
Iteration 3, loss = 0.28286283
Iteration 4, loss = 0.14996988
Iteration 5, loss = 0.10439667


C:\Users\karthikranjan\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Count of particles 1
Iteration 1, loss = 1.44521102
Iteration 2, loss = 0.64109642
Iteration 3, loss = 0.25090579
Iteration 4, loss = 0.13603112
Iteration 5, loss = 0.09972282
Count of particles 2
Iteration 1, loss = 1.46030606
Iteration 2, loss = 0.65509068
Iteration 3, loss = 0.24184183
Iteration 4, loss = 0.13692136
Iteration 5, loss = 0.10245996
Count of particles 3
Iteration 1, loss = 1.45998779
Iteration 2, loss = 0.67679283
Iteration 3, loss = 0.26860677
Iteration 4, loss = 0.14909903
Iteration 5, loss = 0.11098068
Count of particles 4
Iteration 1, loss = 1.43778786
Iteration 2, loss = 0.63090509
Iteration 3, loss = 0.25396419
Iteration 4, loss = 0.14063324
Iteration 5, loss = 0.10066804
Count of particles 5
Iteration 1, loss = 1.42740878
Iteration 2, loss = 0.62034831
Iteration 3, loss = 0.24665380
Iteration 4, loss = 0.12940976
Iteration 5, loss = 0.09144959
Count of particles 6
Iteration 1, loss = 1.46662927
Iteration 2, loss = 0.68975024
Iteration 3, loss = 0.27150816
Iterat

Iteration 3, loss = 0.24823565
Iteration 4, loss = 0.14060084
Iteration 5, loss = 0.10124158
Count of particles 13
Iteration 1, loss = 1.45695700
Iteration 2, loss = 0.66395198
Iteration 3, loss = 0.26447767
Iteration 4, loss = 0.14722339
Iteration 5, loss = 0.10544107
Count of particles 14
Iteration 1, loss = 1.46834110
Iteration 2, loss = 0.69849443
Iteration 3, loss = 0.27497756
Iteration 4, loss = 0.16156106
Iteration 5, loss = 0.11390950
Count of particles 15
Iteration 1, loss = 1.42580170
Iteration 2, loss = 0.59796189
Iteration 3, loss = 0.23930098
Iteration 4, loss = 0.13445215
Iteration 5, loss = 0.10109429
Count of particles 16
Iteration 1, loss = 1.47458669
Iteration 2, loss = 0.70340925
Iteration 3, loss = 0.26735576
Iteration 4, loss = 0.14839740
Iteration 5, loss = 0.10699664
Count of particles 17
Iteration 1, loss = 1.45871267
Iteration 2, loss = 0.64988886
Iteration 3, loss = 0.24623599
Iteration 4, loss = 0.13788551
Iteration 5, loss = 0.10107657
Count of particles 18


INFO:pyswarms.discrete.binary:Iteration 1/10, cost: 0.08261219706524761


...

Iteration 2, loss = 0.57534307
Iteration 3, loss = 0.20247088
Iteration 4, loss = 0.10664063
Iteration 5, loss = 0.08128296
Count of particles 0
Iteration 1, loss = 1.42470377
Iteration 2, loss = 0.54985697
Iteration 3, loss = 0.19061524
Iteration 4, loss = 0.10067036
Iteration 5, loss = 0.07768821
Count of particles 1
Iteration 1, loss = 1.39037294
Iteration 2, loss = 0.52319505
Iteration 3, loss = 0.19219652
Iteration 4, loss = 0.10485696
Iteration 5, loss = 0.07772081
Count of particles 2
Iteration 1, loss = 1.43314759
Iteration 2, loss = 0.59046016
Iteration 3, loss = 0.21110684
Iteration 4, loss = 0.10658729
Iteration 5, loss = 0.07888278
Count of particles 3
Iteration 1, loss = 1.41767828
Iteration 2, loss = 0.55489071
Iteration 3, loss = 0.19794127
Iteration 4, loss = 0.10550728
Iteration 5, loss = 0.08261181
Count of particles 4
Iteration 1, loss = 1.41171903
Iteration 2, loss = 0.56117876
Iteration 3, loss = 0.20344954
Iteration 4, loss = 0.10837439
Iteration 5, loss = 0.07911

Iteration 5, loss = 0.07918327
Count of particles 11
Iteration 1, loss = 1.41112953
Iteration 2, loss = 0.54856684
Iteration 3, loss = 0.18791762
Iteration 4, loss = 0.10486740
Iteration 5, loss = 0.07779712
Count of particles 12
Iteration 1, loss = 1.44862716
Iteration 2, loss = 0.60090770
Iteration 3, loss = 0.20028001
Iteration 4, loss = 0.10295854
Iteration 5, loss = 0.07903131
Count of particles 13
Iteration 1, loss = 1.41473977
Iteration 2, loss = 0.54848421
Iteration 3, loss = 0.19476421
Iteration 4, loss = 0.10618941
Iteration 5, loss = 0.08104550
Count of particles 14
Iteration 1, loss = 1.41749717
Iteration 2, loss = 0.58995781
Iteration 3, loss = 0.20537800
Iteration 4, loss = 0.11093813
Iteration 5, loss = 0.08396977
Count of particles 15
Iteration 1, loss = 1.43108421
Iteration 2, loss = 0.58110461
Iteration 3, loss = 0.20369831
Iteration 4, loss = 0.10801527
Iteration 5, loss = 0.07955532
Count of particles 16
Iteration 1, loss = 1.43253862
Iteration 2, loss = 0.56238306


Iteration 2, loss = 0.63986785
Iteration 3, loss = 0.22798538
Iteration 4, loss = 0.12081281
Iteration 5, loss = 0.08880019
Count of particles 23
Iteration 1, loss = 1.44821578
Iteration 2, loss = 0.61982891
Iteration 3, loss = 0.21690582
Iteration 4, loss = 0.11254755
Iteration 5, loss = 0.08668610
Count of particles 24
Iteration 1, loss = 1.42653927
Iteration 2, loss = 0.57253881
Iteration 3, loss = 0.20737758
Iteration 4, loss = 0.11445304
Iteration 5, loss = 0.08871527
Count of particles 25
Iteration 1, loss = 1.42983836
Iteration 2, loss = 0.58078368
Iteration 3, loss = 0.20816192
Iteration 4, loss = 0.11298995
Iteration 5, loss = 0.08245957
Count of particles 26
Iteration 1, loss = 1.41337572
Iteration 2, loss = 0.57693714
Iteration 3, loss = 0.21232422
Iteration 4, loss = 0.11505870
Iteration 5, loss = 0.08533259
Count of particles 27
Iteration 1, loss = 1.43393741
Iteration 2, loss = 0.57488341
Iteration 3, loss = 0.20095495
Iteration 4, loss = 0.11172510
Iteration 5, loss = 0.

Iteration 4, loss = 0.09715834
Iteration 5, loss = 0.07523044
Count of particles 34
Iteration 1, loss = 1.42806194
Iteration 2, loss = 0.57534307
Iteration 3, loss = 0.20247088
Iteration 4, loss = 0.10664063
Iteration 5, loss = 0.08128296


INFO:pyswarms.discrete.binary:================================
Optimization finished!
Final cost: 0.0691
Best value: [ 1.000000 1.000000 0.000000 ...]



In [11]:
# Time taken
print("Time taken to select features from train data by BPSO", total_time_BPSO)

Time taken to select features from train data by BPSO 30064.671100998727


In [12]:
# Get the selected features from the final positions

text_train = text_train[:,pos==1] # Subset
text_test = text_test[:,pos==1] # Subset

In [13]:
#Shape of selected features subset
print("shape of the selected subset of features by BPSO", text_train.shape[1])

shape of the selected subset of features by BPSO 7207


In [14]:
    # Evaluate hybrid model using multi-layer perceptron (neural network) Classifier
    
    units = int((5 + text_train.shape[1]) / 2)
    
    classifier = Sequential()
    
    #Adding input layer    
    classifier.add(Dense(1024, input_shape=(text_train.shape[1],), kernel_initializer ='uniform', activation='relu'))
    
    # Adding the second hidden layer
    classifier.add(Dense(512, kernel_initializer = 'uniform', activation = 'relu'))
    
    # Adding the output layer
    #classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
    classifier.add(Dense(5, kernel_initializer = 'uniform', activation = 'sigmoid'))
    
    classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
   # Fitting the ANN to the Training set
classifier.fit(text_train, sent_train, batch_size = 100, nb_epoch = 10)

y_pred_pso = classifier.predict(text_test)

score = classifier.evaluate(text_test, sent_test,
                       batch_size=100, verbose=1)

print('Test accuracy:', score[1])

C:\Users\karthikranjan\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/10
3591/3591 [==============================] - 8s 2ms/step - loss: 1.4063 - acc: 0.5141
Epoch 2/10
3591/3591 [==============================] - 7s 2ms/step - loss: 0.4291 - acc: 0.8580
Epoch 3/10
3591/3591 [==============================] - 7s 2ms/step - loss: 0.1445 - acc: 0.9507
Epoch 4/10
3591/3591 [==============================] - 7s 2ms/step - loss: 0.0864 - acc: 0.9708
Epoch 5/10
3591/3591 [==============================] - 7s 2ms/step - loss: 0.0708 - acc: 0.9727
Epoch 6/10
3591/3591 [==============================] - 7s 2ms/step - loss: 0.0711 - acc: 0.9722
Epoch 7/10
3591/3591 [==============================] - 7s 2ms/step - loss: 0.0659 - acc: 0.9738
Epoch 8/10
3591/3591 [==============================] - 7s 2ms/step - loss: 0.0638 - acc: 0.9730A: 4s - l
Epoch 9/10
3591/3591 [==============================] - 7s 2ms/step - loss: 0.0650 - acc: 0.9719
Epoch 10/10
898/898 [==============================] - 0s 313us/step
Test accuracy: 0.8396436511009998


In [15]:
#classification report matrix for hybrid model
from sklearn import metrics

y_class_pred_pso = np.argmax(y_pred_pso, axis=1)

print("Result: Classification performance report of hybrid model(BPSO):\n ", metrics.classification_report(sent_test, y_class_pred_pso))

Result: Classification performance report of hybrid model(BPSO):
               precision    recall  f1-score   support

          0       0.87      0.86      0.87       190
          1       0.93      0.85      0.88       213
          2       0.72      0.90      0.80       198
          3       0.88      0.80      0.84       173
          4       0.83      0.75      0.79       124

avg / total       0.85      0.84      0.84       898



In [16]:

# Get the selected features from the final positions for entire data
X_selected_features = X[:,pos==1]  # subset

print(X_selected_features.shape)

(4489, 7207)


In [20]:
#### K- fold cross validation for hybrid model ##########

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(Dense(units=1024, activation='relu', input_shape=(7207,)))

    # Add fully connected layer with a ReLU activation function
    network.add(Dense(units=512, activation='relu'))

    # Add fully connected layer with a sigmoid activation function
    network.add(Dense(units=5, activation='sigmoid'))

    # Compile neural network
    network.compile(loss='sparse_categorical_crossentropy', # Cross-entropy
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['accuracy']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [21]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
# Wrap Keras model so it can be used by scikit-learn
neural_network = KerasClassifier(build_fn=create_network, 
                                 epochs=5, 
                                 batch_size=100, 
                                 verbose=1)

In [22]:
# Evaluate neural network using five-fold cross-validation - Results
print("5-fold cross validation of hybrid model Results: /n")
val_scores = cross_val_score(neural_network, X_selected_features, y, cv=5)

print("5 fold cross validaton scores with BPSO: ", val_scores)

print("5 fold Average - cross validaton scores with BPSO: ", np.average(val_scores))

5-fold cross validation of hybrid model Results: /n
Epoch 1/5
3591/3591 [==============================] - 8s 2ms/step - loss: 1.2369 - acc: 0.6344
Epoch 2/5
3591/3591 [==============================] - 6s 2ms/step - loss: 0.2911 - acc: 0.9006
Epoch 3/5
3591/3591 [==============================] - 6s 2ms/step - loss: 0.1111 - acc: 0.9616A: 1s - loss: 0.1056 - a
Epoch 4/5
3591/3591 [==============================] - 6s 2ms/step - loss: 0.0789 - acc: 0.9671
Epoch 5/5
898/898 [==============================] - 0s 280us/step
Epoch 1/5
3591/3591 [==============================] - 7s 2ms/step - loss: 1.2515 - acc: 0.5954
Epoch 2/5
3591/3591 [==============================] - 6s 2ms/step - loss: 0.2886 - acc: 0.9000
Epoch 3/5
3591/3591 [==============================] - 6s 2ms/step - loss: 0.1118 - acc: 0.9568A: 4s - l
Epoch 4/5
3591/3591 [==============================] - 6s 2ms/step - loss: 0.0803 - acc: 0.9702
Epoch 5/5
898/898 [==============================] - 0s 264us/step
Epoch 1/5
359